In [1]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS, add_constant

In [2]:
xls = pd.ExcelFile('./bank_datastream0816.xlsx')
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df.to_csv(f'{sheet_name}.csv', index=False)
xls = pd.ExcelFile('./Europe_3_Factors_Daily.xlsx')
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df.to_csv(f'{sheet_name}.csv', index=False)

In [3]:
stock_data=pd.read_csv("./share price daily.csv")

ff_factors = pd.read_csv("./Europe_3_Factors_Daily_stoxx.csv")

# ff_factors = pd.read_csv("./Europe_3_Factors_Daily_sp.csv")

riskfree_data = pd.read_csv("./RF.csv")


In [4]:
stock_data.head()

,Date,EBS,OBS.VI,RBI.VI,ARLN.HM,CBK.DE,PBB.BE,MBK.DE,BBVA.MC,UNIE.XC,...,ISP.MI,UCG.MI,ABN.AS,VLK.AS,AZA.ST,HOFI.ST,SEB-A.ST,SHB-A.ST,TFBANK.ST,NLB.IL
0,2019-01-02,57.700001,42.317120,15.009684,20.131157,5.346076,4.841865,7.301744,3.424062,0.962672,...,1.291749,8.020234,12.378850,10.646761,69.065323,42.700001,70.079155,74.827446,68.548180,9.270435
1,2019-01-03,57.540001,42.411366,15.284290,20.079140,5.380388,4.863699,7.260255,3.433240,0.958453,...,1.288361,7.906972,12.481354,10.517239,69.483315,42.849998,70.615601,74.888100,70.479111,9.224634
2,2019-01-04,58.790001,42.411366,15.706250,20.041981,5.696608,4.858240,7.260255,3.561362,0.968578,...,1.354629,8.378076,12.855190,10.672668,70.962372,43.099998,72.615067,76.419518,71.637665,9.296197
3,2019-01-07,61.549999,42.411366,16.000950,20.651340,5.732775,5.092965,7.343231,3.548881,0.983765,...,1.377532,8.599673,12.963724,10.776285,71.219597,45.349998,71.265839,75.600746,71.444580,9.039998
4,2019-01-08,62.700001,42.411366,16.027742,21.008039,5.870019,5.142092,7.260255,3.578984,0.976171,...,1.361609,8.458508,12.945635,10.750380,70.126381,44.450001,72.029877,75.085220,72.410034,9.031410


In [5]:
ff_factors

,Date,Mkt-RF,SMB,HML
0,2019-01-02,0.368000,0.55,-0.22
1,2019-01-03,0.719131,0.57,0.80
2,2019-01-04,0.759509,-0.93,-0.06
3,2019-01-07,0.727301,1.21,-0.43
4,2019-01-08,0.731021,-0.01,-0.50
...,...,...,...,...
506,2020-12-24,0.941136,0.39,-0.03
507,2020-12-28,0.945068,-0.33,-0.31
508,2020-12-29,0.945667,0.22,-0.84
509,2020-12-30,0.937269,0.30,-0.08


In [6]:
riskfree_data

,Date,RF
0,2019-01-02,-0.368
1,2019-01-03,-0.366
2,2019-01-04,-0.365
3,2019-01-07,-0.365
4,2019-01-08,-0.362
...,...,...
506,2020-12-24,-0.470
507,2020-12-28,-0.468
508,2020-12-29,-0.472
509,2020-12-30,-0.470


In [7]:
# Parameters
event_date = '2020-03-27'
estimation_window_start = -63  # Start of the estimation window
estimation_window_end = -4     # End of the estimation window
event_window_start = -3  #  days before the event
event_window_end = 3     #  days after the event
bankname = stock_data.iloc[0:0,1:]
bank_list = bankname.columns[0:].tolist()  # Convert the first row of stock_data to a list

In [8]:
print(bank_list)

['EBS', 'OBS.VI', 'RBI.VI', 'ARLN.HM', 'CBK.DE', 'PBB.BE', 'MBK.DE', 'BBVA.MC', 'UNIE.XC', 'AKTIA.HE', 'NDA-SE.ST', 'OMASP.HE', 'BNP.PA', 'CCN.PA', 'LLOY.L', 'MTRO.L', 'STB.L', 'STAN.L', 'TATT.AT', 'BIRG.IR', 'ARION.IC', 'BGN.MI', 'BMED.MI', 'BDB.MI', 'FBK.MI', 'ILTY.MI', 'ISP.MI', 'UCG.MI', 'ABN.AS', 'VLK.AS', 'AZA.ST', 'HOFI.ST', 'SEB-A.ST', 'SHB-A.ST', 'TFBANK.ST', 'NLB.IL']


In [9]:
# Clean up column names
stock_data.columns = stock_data.columns.str.strip()
riskfree_data.columns = riskfree_data.columns.str.strip()

# Convert 'Date' column to datetime
date_column_name = 'Date'
stock_data[date_column_name] = pd.to_datetime(stock_data[date_column_name])
riskfree_data[date_column_name] = pd.to_datetime(riskfree_data[date_column_name])

# Set Date as index if it's not already
stock_data.set_index(date_column_name, inplace=True)
riskfree_data.set_index(date_column_name, inplace=True)


In [10]:
# Initialize an empty DataFrame to store daily returns
bank_daily_returns = pd.DataFrame()

selected_metric = 'RF'
riskfree_data_series = pd.to_numeric(riskfree_data[selected_metric], errors='coerce')
    
# Iterate over each column to calculate daily returns
for column in stock_data.columns:
    if column != date_column_name:  # Skip the Date column
        # Ensure the column data is numeric
        stock_data[column] = pd.to_numeric(stock_data[column], errors='coerce')
        
        # Calculate daily returns for this column
        bank_daily_returns[column] = stock_data[column].pct_change()
      
        # Calculate excess returns by subtracting the risk-free rate
        bank_daily_returns[column] = bank_daily_returns[column] - riskfree_data_series

# If the Date column was reset earlier, there's no need to add it back again
if date_column_name not in bank_daily_returns.columns:
    bank_daily_returns.reset_index(inplace=True)

# Reorder columns so that Date appears first
bank_daily_returns = bank_daily_returns[[date_column_name] + [col for col in bank_daily_returns.columns if col != date_column_name]]


In [11]:
print(bank_daily_returns)

          Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
0   2019-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
1   2019-01-03  0.363227  0.368227  0.384295  0.363416  0.372418  0.370509   
2   2019-01-04  0.386724  0.365000  0.392607  0.363149  0.423773  0.363878   
3   2019-01-07  0.411947  0.365000  0.383763  0.395404  0.371349  0.413315   
4   2019-01-08  0.380684  0.362000  0.363674  0.379272  0.385940  0.371646   
..         ...       ...       ...       ...       ...       ...       ...   
506 2020-12-24  0.457166  0.470000  0.470000  0.470000  0.470000  0.470000   
507 2020-12-28  0.450097  0.468000  0.469186  0.481882  0.467627  0.448833   
508 2020-12-29  0.450082  0.472000  0.469038  0.473521  0.457075  0.470883   
509 2020-12-30  0.498400  0.472375  0.461087  0.459873  0.467348  0.461615   
510 2020-12-31  0.464559  0.498000  0.498000  0.498000  0.498000  0.498000   

       MBK.DE   BBVA.MC   UNIE.XC  ...    ISP.MI    UCG.MI    A

In [12]:
ff_factors['Date'] = pd.to_datetime(ff_factors['Date'])
# Merge the two DataFrames on the Data column
merged_data=pd.merge(bank_daily_returns, ff_factors, on=date_column_name, suffixes=('_stock', '_market'))
print(merged_data)

          Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
0   2019-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
1   2019-01-03  0.363227  0.368227  0.384295  0.363416  0.372418  0.370509   
2   2019-01-04  0.386724  0.365000  0.392607  0.363149  0.423773  0.363878   
3   2019-01-07  0.411947  0.365000  0.383763  0.395404  0.371349  0.413315   
4   2019-01-08  0.380684  0.362000  0.363674  0.379272  0.385940  0.371646   
..         ...       ...       ...       ...       ...       ...       ...   
506 2020-12-24  0.457166  0.470000  0.470000  0.470000  0.470000  0.470000   
507 2020-12-28  0.450097  0.468000  0.469186  0.481882  0.467627  0.448833   
508 2020-12-29  0.450082  0.472000  0.469038  0.473521  0.457075  0.470883   
509 2020-12-30  0.498400  0.472375  0.461087  0.459873  0.467348  0.461615   
510 2020-12-31  0.464559  0.498000  0.498000  0.498000  0.498000  0.498000   

       MBK.DE   BBVA.MC   UNIE.XC  ...    VLK.AS    AZA.ST   HO

In [13]:
def find_data_in_relevent_days(origin_data, start, end):
    res_index = origin_data.index[origin_data['Date'] == event_date].tolist()[0]
    res_index_position = origin_data.index.get_loc(res_index) # true index: 3
    res_data = origin_data.iloc[res_index_position+start: res_index_position+end+1]
    return res_data


estimation_data = find_data_in_relevent_days(merged_data, start=estimation_window_start, end=estimation_window_end)
event_data = find_data_in_relevent_days(merged_data, start=event_window_start, end=event_window_end)

print(estimation_data)
print(event_data)



          Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
253 2019-12-30  0.461370  0.453917  0.448465  0.450450  0.469596  0.462216   
254 2019-12-31  0.439737  0.446000  0.446000  0.446000  0.446000  0.446000   
255 2020-01-02  0.454927  0.451912  0.477671  0.442510  0.522515  0.465668   
256 2020-01-03  0.456444  0.447816  0.441529  0.479566  0.434188  0.440467   
257 2020-01-06  0.467459  0.451899  0.444300  0.446243  0.422739  0.451941   
258 2020-01-07  0.438809  0.448684  0.442533  0.483664  0.472294  0.472194   
259 2020-01-08  0.469121  0.451000  0.456410  0.435167  0.472381  0.453704   
260 2020-01-09  0.483422  0.454000  0.464762  0.470731  0.457946  0.455349   
261 2020-01-10  0.435710  0.457237  0.457437  0.457747  0.436251  0.441552   
262 2020-01-13  0.431354  0.454110  0.444049  0.449165  0.438094  0.434970   
263 2020-01-14  0.465832  0.457210  0.456117  0.432785  0.454763  0.459930   
264 2020-01-15  0.447751  0.455096  0.430363  0.405611  0.413584

In [14]:
# Fama-French 3 factor model
# Perform OLS regression for each bank against the market indices
results = {}
cumulative_abnormal_returns = pd.DataFrame()
factor_columns = ['Mkt-RF', 'SMB', 'HML']

for bank in bank_list:
    X = estimation_data[factor_columns]  # Independent variables (3 factors)
    X = add_constant(X)  # Add a constant (intercept)
    y = estimation_data[bank]  # Dependent variable (bank returns)
    
    
    model = OLS(y, X).fit()  # Fit the model
    
#     print(model.summary())
    
#     results = {}
    
    # Predict expected returns during the event window
    event_data[f'Expected_{bank}'] = model.predict(add_constant(event_data[factor_columns]))
    
    # Calculate abnormal returns
    event_data[f'Abnormal_{bank}'] = event_data[bank] - event_data[f'Expected_{bank}']
    
    # Calculate cumulative abnormal returns (CAR)
    event_data[f'CAR_{bank}'] = event_data[f'Abnormal_{bank}'].cumsum()
    
    # Save CAR
    cumulative_abnormal_returns[f'CAR_{bank}'] = event_data[f'CAR_{bank}']
    
    print(f"Abnormal Returns (AR) for {bank}:")
    print(event_data[f'Abnormal_{bank}'])

    
# Calculate the CAAR (Cumulative Average Abnormal Returns)
cumulative_abnormal_returns['CAAR'] = cumulative_abnormal_returns.mean(axis=1)

    
#     # Print the summary for each bank
#     print(f"Regression results for {bank}:")
#     print(model.summary())
#     print("\n" + "="*80 + "\n")
    
    # Display the results
# print("Estimation Start Date:", estimation_start_date)
# print("Estimation End Date:", estimation_end_date)


Abnormal Returns (AR) for EBS:
313    0.032564
314   -0.050827
315    0.045728
316   -0.077752
317    0.090547
318    0.040345
319   -0.042666
Name: Abnormal_EBS, dtype: float64
Abnormal Returns (AR) for OBS.VI:
313   -0.005070
314   -0.010204
315   -0.006285
316   -0.007723
317   -0.000947
318   -0.019962
319   -0.006533
Name: Abnormal_OBS.VI, dtype: float64
Abnormal Returns (AR) for RBI.VI:
313    0.021822
314   -0.018748
315    0.050504
316    0.003210
317   -0.026289
318   -0.030146
319    0.014563
Name: Abnormal_RBI.VI, dtype: float64
Abnormal Returns (AR) for ARLN.HM:
313    0.005368
314    0.135806
315   -0.072144
316    0.077003
317    0.007893
318    0.014487
319    0.055919
Name: Abnormal_ARLN.HM, dtype: float64
Abnormal Returns (AR) for CBK.DE:
313   -0.056459
314    0.017539
315    0.027829
316    0.030728
317   -0.036412
318   -0.003469
319    0.011842
Name: Abnormal_CBK.DE, dtype: float64
Abnormal Returns (AR) for PBB.BE:
313    0.049649
314    0.055617
315   -0.011458
31

/Users/klara/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_47272/882673880.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_data[f'Expected_{bank}'] = model.predict(add_constant(event_data[factor_columns]))
/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_47272/882673880.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

Abnormal Returns (AR) for BNP.PA:
313   -0.031499
314    0.053051
315    0.033520
316    0.036880
317   -0.030120
318   -0.019337
319    0.007562
Name: Abnormal_BNP.PA, dtype: float64
Abnormal Returns (AR) for CCN.PA:
313   -0.044140
314    0.024854
315   -0.002040
316    0.033461
317    0.007754
318   -0.002513
319   -0.003369
Name: Abnormal_CCN.PA, dtype: float64
Abnormal Returns (AR) for LLOY.L:
313    0.020373
314    0.073666
315    0.025624
316   -0.039325
317   -0.003196
318   -0.046278
319   -0.074667
Name: Abnormal_LLOY.L, dtype: float64
Abnormal Returns (AR) for MTRO.L:
313   -0.023125
314    0.150137
315    0.056161
316   -0.069727
317    0.016760
318   -0.015583
319   -0.010794
Name: Abnormal_MTRO.L, dtype: float64
Abnormal Returns (AR) for STB.L:
313   -0.071756
314    0.115847
315   -0.045052
316    0.189149
317    0.005665
318   -0.014413
319    0.040800
Name: Abnormal_STB.L, dtype: float64
Abnormal Returns (AR) for STAN.L:
313   -0.025201
314    0.030932
315    0.013190


In [15]:
# Display CAAR for the event window
print(cumulative_abnormal_returns['CAAR'])

313   -0.016245
314    0.001170
315    0.016309
316    0.029276
317    0.025949
318    0.032373
319    0.046301
Name: CAAR, dtype: float64


In [16]:
print(event_data[['Date'] +[f'CAR_{bank}' for bank in bank_list]])
    

          Date   CAR_EBS  CAR_OBS.VI  CAR_RBI.VI  CAR_ARLN.HM  CAR_CBK.DE  \
313 2020-03-24  0.032564   -0.005070    0.021822     0.005368   -0.056459   
314 2020-03-25 -0.018262   -0.015274    0.003074     0.141174   -0.038920   
315 2020-03-26  0.027465   -0.021559    0.053578     0.069030   -0.011092   
316 2020-03-27 -0.050287   -0.029282    0.056788     0.146033    0.019636   
317 2020-03-30  0.040260   -0.030229    0.030500     0.153926   -0.016776   
318 2020-03-31  0.080605   -0.050192    0.000354     0.168414   -0.020245   
319 2020-04-01  0.037939   -0.056724    0.014917     0.224332   -0.008403   

     CAR_PBB.BE  CAR_MBK.DE  CAR_BBVA.MC  CAR_UNIE.XC  ...  CAR_ISP.MI  \
313    0.049649   -0.058110    -0.024554    -0.099454  ...   -0.040473   
314    0.105266   -0.084198     0.000984    -0.088362  ...   -0.051125   
315    0.093808   -0.100223     0.006967    -0.083602  ...   -0.043368   
316    0.094308   -0.137788     0.026658    -0.020962  ...   -0.011854   
317    0.0815

In [17]:
from scipy import stats
from scipy.stats import t

windows = {
    'CAR [-1,+1]': [-1, 1],
    'CAR [-1,+3]': [-1, 3],
    'CAR [0,+1]': [0, 1],
}


car_results = pd.DataFrame()
# Assume event_data contains the columns: 'bank', 'days_from_event', and 'Abnormal_Return'

for label, (start, end) in windows.items():
    # Filter data for the specified time window
#     print("event_data:", event_data)
    window_data = find_data_in_relevent_days(event_data, start=start, end=end)

    print(f"{label}-window_data:", window_data)
    
#     assert window_data.equals(window_data2)
    
    # Calculate CAR for each bank
    car_per_bank = {}
    for bank in bank_list:
        if bank in window_data.columns:
            car_per_bank[bank] = window_data[f'Abnormal_{bank}'].sum()

    # Convert to DataFrame for merging
    car_per_bank_df = pd.DataFrame(list(car_per_bank.items()), columns=['Bank', label])
    
    # Merge the results into the car_results DataFrame
    if car_results.empty:
        car_results = car_per_bank_df
    else:
        car_results = pd.merge(car_results, car_per_bank_df, on='Bank', how='outer')
        
# Convert CAR results to percentages and format them to two decimal places
for column in car_results.columns[1:]:  # Skip the 'Bank' column
    car_results[column] = car_results[column] * 100  # Convert to percentage
    car_results[column] = car_results[column].apply(lambda x: f"{x:.2f}%")  # Format to two decimal places

# Display the calculated CAR results for each company
print(car_results)


CAR [-1,+1]-window_data:           Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
315 2020-03-26  0.491880  0.444000  0.496780  0.374594  0.471093  0.432316   
316 2020-03-27  0.338268  0.449819  0.405227  0.459381  0.385945  0.416488   
317 2020-03-30  0.507037  0.443000  0.399840  0.438714  0.374102  0.410073   

       MBK.DE   BBVA.MC   UNIE.XC  ...  CAR_SEB-A.ST  Expected_SHB-A.ST  \
315  0.432235  0.448903  0.446913  ...     -0.017025           0.454686   
316  0.409286  0.391638  0.454681  ...     -0.006760           0.390346   
317  0.467691  0.383570  0.469846  ...     -0.007090           0.455924   

     Abnormal_SHB-A.ST  CAR_SHB-A.ST  Expected_TFBANK.ST  Abnormal_TFBANK.ST  \
315          -0.003665      0.017466            0.452881           -0.017452   
316           0.018400      0.035865            0.407266           -0.008376   
317          -0.051747     -0.015882            0.453319           -0.016361   

     CAR_TFBANK.ST  Expected_NLB.IL  Abn

In [18]:
import numpy as np
from scipy.stats import t

# Create a copy of car_results to annotate the CAR values
car_results_copy = car_results.copy()

# Define significance levels and markers
alpha_levels = [0.01, 0.05, 0.10]
markers = ["***", "**", "*"]

# Iterate over each bank and window to annotate CAR values with significance markers
for bank in car_results['Bank'].unique():
    for label, (start, end) in windows.items():
        # Get abnormal returns for the specified window
        window_data = find_data_in_relevent_days(event_data, start=start, end=end)

        if f'Abnormal_{bank}' in window_data.columns:
            abnormal_returns = window_data[f'Abnormal_{bank}']

            # Calculate the standard deviation of abnormal returns
            std_dev = abnormal_returns.std(ddof=1)

            # Calculate the number of observations (days in the event window)
            n = len(abnormal_returns)

            if n > 1 and std_dev > 0:
                # Calculate the standard error (SE)
                standard_error = std_dev / np.sqrt(n)

                # Get the CAR value (removing percentage sign and converting to decimal)
                car_value = float(car_results.loc[car_results['Bank'] == bank, label].str.rstrip('%')) / 100

                # Calculate the t-statistic
                t_stat = car_value / standard_error

                # Annotate CAR value with significance markers based on different levels
                annotation = ""
                for alpha, marker in zip(alpha_levels, markers):
                    # Calculate the critical t-value for a two-tailed test
                    critical_value = t.ppf(1 - alpha / 2, df=n - 1)

                    # Determine if the t-statistic is greater than the critical value (significant)
                    if abs(t_stat) > critical_value:
                        annotation = marker
                        break  # Break at the highest significance level

                # Update the CAR value in the copy with the appropriate significance marker
                if annotation:
                    current_value = car_results_copy.loc[car_results_copy['Bank'] == bank, label].values[0]
                    car_results_copy.loc[car_results_copy['Bank'] == bank, label] = f"{current_value}{annotation}"
            else:
                # Detailed message for skipping due to insufficient data or zero variance
                if n <= 1:
                    print(f"Not enough observations for bank {bank} in window {label}. Skipping t-test.")
                elif std_dev == 0:
                    print(f"Zero variance for bank {bank} in window {label}. Skipping t-test.")
        else:
            print(f"No data available for bank {bank} in window {label}. Skipping t-test.")

# Display the annotated CAR results with significance markers
print("CAR Results with Significance Annotations:")
print(car_results_copy)


CAR Results with Significance Annotations:
         Bank CAR [-1,+1] CAR [-1,+3] CAR [0,+1]
0         EBS       5.85%       5.62%      1.28%
1      OBS.VI    -1.50%**   -4.15%***     -0.87%
2      RBI.VI       2.74%       1.18%     -2.31%
3     ARLN.HM       1.28%     8.32%**      8.49%
4      CBK.DE       2.21%      3.05%*     -0.57%
5      PBB.BE    -2.37%**      -0.48%     -1.23%
6      MBK.DE      -1.18%      -2.65%      0.42%
7     BBVA.MC      -0.52%      -0.52%     -1.12%
8     UNIE.XC    12.14%**     7.86%**   11.66%**
9    AKTIA.HE     3.75%**    8.16%***      2.44%
10  NDA-SE.ST       2.23%    7.15%***      0.93%
11   OMASP.HE       0.19%       1.13%     -4.11%
12     BNP.PA       4.03%       2.85%      0.68%
13     CCN.PA      3.92%*    3.33%***      4.12%
14     LLOY.L      -1.69%  -13.78%***     -4.25%
15     MTRO.L       0.32%      -2.32%     -5.30%
16      STB.L      14.98%    17.61%**     19.48%
17     STAN.L       2.32%      -1.22%      1.00%
18    TATT.AT       6.47% 

In [19]:
# output_file_path = './3factor_CAR_Results_250_stoxx.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path

In [20]:
# output_file_path = './3factor_CAR_Results_250_sp.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path

In [21]:
output_file_path = './3factor_CAR_Results_63_stoxx.xlsx'
car_results_copy.to_excel(output_file_path, index=False)

output_file_path

'./3factor_CAR_Results_63_stoxx.xlsx'

In [22]:
# output_file_path = './3factor_CAR_Results_63_sp.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path